In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, HTML

import marshaltools

In [ ]:
import requests
import re

def check_status(code):
    if code != 200:
        if code in marshaltools.gci_utils.httpErrors.keys():
            raise ValueError(marshaltools.gci_utils,httpErrors[code])
        else:
            raise ValueError('Error %i'%code)

def iniatiate_transfer(name, to_program=32):
    # Get sourceid
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi', 
                      auth=(pl1.user, pl1.passwd), 
                      data={'name': name})
    check_status(r.status_code)            
    sourceid = re.findall('[0-9]+', re.findall('transfer_source.cgi\?sourceid=[0-9]+', r.text)[0])[0]
    
    # id of from_program
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                  auth=(pl1.user, pl1.passwd), 
                  data={'sourceid': sourceid})
    check_status(r.status_code)            
    programid = re.findall('[0-9]+', re.findall('From Program: <select name=from_program >\n<option value=\"[0-9]+\">', r.text)[0])[0]

    # create transfer
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                  auth=(pl1.user, pl1.passwd), 
                  data={'commit': 'yes', 'sourceid': sourceid, 'id': -1, 'from_program': programid,
                        'to_program': 32, 'share': 'yes', 'comment': '', 'status': 'Submit'})
    check_status(r.status_code)            
    
    return sourceid

def confirm_transfer(sourceids):
    # get currently pending transactions to determine transaction id
    r = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/pending_transactions.cgi', 
                      auth=(pl1.user, pl1.passwd))
    check_status(r.status_code)    
    
    for name, sourceid in sourceids.items():
        pattern = '<input type=hidden name="id" value=[0-9]+>\s*<input type=hidden name="sourceid" value=%s>'%sourceid
        query = re.findall(pattern, r.text)
        
        if len(query) > 0:
            transid = re.findall('[0-9]+', query[0])[0]
    
            r2 = requests.post('http://skipper.caltech.edu:8080/cgi-bin/growth/transfer_source.cgi', 
                              auth=(pl1.user, pl1.passwd), 
                              data={'commit': 'yes', 'sourceid': sourceid, 'id': transid, 'status': 'Accept'})
        else:
            print('%s (id %s) not found in transactions.'%(name, sourceid))
        

In [ ]:
# Start by loading the list of SNe for the Cosmology program
pl1 = marshaltools.ProgramList('Cosmology')

In [ ]:
pls = {}

In [ ]:
# Furthermore load a list of all source on the marshal that you have access to.
# This may take a while.
for p in pl1.program_list:
    if p['name'] != 'Cosmology' and p['name'] not in pls.keys():
        pls[p['name']] = marshaltools.ProgramList(p['name'], timeout=300) 

In [ ]:
# Figure out which SNe should be added to the Cosmology program
new = []
for pl_ in pls.values():
    for s_ in pl_.sources.values():
        if (s_['name'] not in pl1.sources.keys() 
            and (s_['name'], s_['classification']) not in new
            and s_['classification'] is not None 
            and s_['classification'].startswith('SN Ia')):
            new.append((s_['name'], s_['classification']))
            
print('Number of SNe Ia currently in the Cosmology program:', 
      len([a for a in pl1.sources.values() 
      if a['classification'] is not None and
      a['classification'].startswith('SN Ia')]))
print('Number of SNe Ia to be added:', len(new))

In [ ]:
# Add them in groups of 10, i.e. 10 SNe are added to the transfer page and then confirmed.
# This saves a few HTML requests. Do not set n to a much higher value because this will add
# a lot of SNe to the transfer page, making it hard to use for a few minutes, best to avoid that.

n = 10

while len(new) > 0: 
    sourceids = {}
    for k in range(min(len(new), n)):
        name = new.pop(0)[0]
        print(name)
    
        sourceids[name] = iniatiate_transfer(name)

    confirm_transfer(sourceids)
    print('Done.')

In [ ]:
print('Number of SNe Ia now in the Cosmology program:', 
      len([a for a in pl1.sources.values() 
      if a['classification'] is not None and
      a['classification'].startswith('SN Ia')]))